Design of a new game, Spuds

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
end

const MXS = 100
const MXV = 10
const MNV = 1

function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
    #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
    return h*(f+l) + f*(p+r+s)
end

cost (generic function with 1 method)

In [3]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end

function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, utb)
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    end
end

eval_battle (generic function with 1 method)

In [4]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)

const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat()::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    adj = adjectives[adj_i, :adjective]
    vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
    vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
    vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
    vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
    vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
    vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6])
end

random_name_and_stat (generic function with 1 method)

In [5]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s)
end

rand_rename (generic function with 2 methods)

In [6]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s])
    end
    return as
end

function cost(a::Spud)::Int64
    return cost(a.h, a.f, a.l, a.p, a.r, a.s)
end

cost (generic function with 2 methods)

## Form library randomly

In [7]:
# form initial library by subsampling indices
library = Array{Spud}(undef, 100000)
spud_i = 0
ss_prob = 1.1

1.1

In [8]:

hrange = MNV:MXV
frange = MNV:MXV
lrange = MNV:MXV
prange = MNV:MXV
rrange = MNV:MXV
srange = MNV:MXV

for h in hrange
    if (cost(h, MNV, MNV, MNV, MNV, MNV) <= MXS)
        for f in frange
            if (cost(h, f, MNV, MNV, MNV, MNV) <= MXS)
                for l in lrange
                    if (cost(h, f, l, MNV, MNV, MNV) <= MXS)
                        for p in prange
                            if (cost(h, f, l, p, MNV, MNV) <= MXS)
                                for r in rrange
                                    if (cost(h, f, l, p, r, MNV) <= MXS)
                                        for s in srange
                                            if rand() < ss_prob && (cost(h,f,l,p,r,s) <= MXS)
                                                check_f = (h == MXV) || (cost(h+1,f,l,p,r,s) > MXS)
                                                check_h = (f == MXV) || (cost(h,f+1,l,p,r,s) > MXS)
                                                check_l = (l == MXV) || (cost(h,f,l+1,p,r,s) > MXS)
                                                check_prs = (p+r+s == 3*MXV) || (cost(h,f,l,p+1,r,s) > MXS)
                                                if check_h && check_f && check_l && check_prs
                                                    spud_i += 1
                                                    #randname = rand_rename(Spud(" ",h,f,l,p,r,s)).name
                                                    #name = string("#", @sprintf("%i", spud_i), ". ", randname)
                                                    name = ""
                                                    library[spud_i] = Spud(name,h,f,l,p,r,s)
                                                end
                                            end
                                        end
                                    end
                                end                        
                            end
                        end                        
                    end
                end
            end
        end
    end
end


In [9]:
library = unique(library[1:spud_i])
n_spuds = length(library)

12675

In [230]:
nash_env_df = DataFrame(CSV.File("spudsD_mxv10_nash.csv"))
nash_env = df_to_spuds(nash_env_df)
counts = [parse(Int, s.name[2:end]) for s in nash_env];

In [231]:
nash_env = nash_env[1:40]

40-element Vector{Spud}:
 Spud("c12207143", 6, 1, 10, 10, 10, 10)
 Spud("c4458137", 3, 2, 10, 10, 10, 10)
 Spud("c3701536", 9, 7, 1, 2, 1, 1)
 Spud("c3597175", 4, 2, 10, 10, 10, 6)
 Spud("c1595416", 7, 1, 10, 10, 3, 10)
 Spud("c1400790", 10, 4, 3, 2, 1, 4)
 Spud("c1267092", 3, 9, 3, 3, 3, 1)
 Spud("c1265702", 2, 3, 10, 10, 10, 4)
 Spud("c1244219", 6, 8, 2, 1, 3, 1)
 Spud("c1228093", 8, 7, 1, 1, 2, 2)
 Spud("c1195209", 3, 7, 3, 5, 1, 4)
 Spud("c1158556", 10, 1, 8, 4, 1, 5)
 Spud("c1137473", 2, 10, 10, 2, 1, 3)
 ⋮
 Spud("c779926", 5, 9, 2, 1, 2, 2)
 Spud("c744203", 3, 8, 1, 5, 2, 2)
 Spud("c723169", 7, 5, 1, 1, 5, 5)
 Spud("c703405", 10, 1, 8, 1, 3, 6)
 Spud("c700603", 3, 8, 1, 4, 4, 1)
 Spud("c687883", 4, 8, 1, 4, 1, 3)
 Spud("c678472", 4, 3, 10, 6, 1, 9)
 Spud("c656057", 10, 1, 8, 2, 4, 4)
 Spud("c654570", 10, 2, 6, 2, 3, 5)
 Spud("c646977", 2, 10, 10, 1, 3, 2)
 Spud("c639140", 9, 1, 9, 5, 1, 4)
 Spud("c637598", 8, 1, 10, 6, 3, 3)

In [232]:
n_nash = length(nash_env)

40

In [233]:
payoffs = Array{Int64}(undef, (n_nash, n_nash))
for i in 1:n_nash
    for j in 1:n_nash
        payoffs[i, j] = eval_battle(nash_env[i], nash_env[j])
    end
end

In [234]:
payoffs

40×40 Matrix{Int64}:
  0  -1  -1  -1   1   1  -1  -1  -1  …   1  -1  -1   1   1   1  -1   1   1
  1   0  -1   1   1  -1  -1   1  -1      1  -1  -1   1   1   1   1   1   1
  1   1   0   1  -1  -1  -1   1   1     -1   1   1   1  -1  -1  -1  -1  -1
  1  -1  -1   0   1   1  -1   1  -1      1  -1  -1   1   1   1   1   1   1
 -1  -1   1  -1   0   1  -1  -1  -1      1  -1  -1   1   1   1  -1   1   1
 -1   1   1  -1  -1   0   1   1   1  …  -1   1   1  -1  -1  -1   1  -1  -1
  1   1   1   1   1  -1   0  -1   1     -1  -1  -1  -1  -1  -1  -1  -1   1
  1  -1  -1  -1   1  -1   1   0  -1      1   1  -1  -1   1  -1   1   1   1
  1   1  -1   1   1  -1  -1   1   0     -1  -1  -1   1  -1  -1  -1  -1  -1
  1   1   1   1  -1  -1  -1   1   1     -1   1  -1   1  -1  -1  -1  -1  -1
  1   1  -1   1  -1  -1   1  -1   1  …  -1   1   1  -1  -1  -1   1  -1  -1
 -1  -1   1  -1  -1   1   1  -1   1     -1   1   1  -1   1   1  -1  -1  -1
  1  -1   1  -1   1  -1  -1  -1   1      1  -1  -1  -1   1  -1   1   1   1
  ⋮ 

## Attempt to use Gaussian elimination

In [235]:
#var_order = reverse(1:n_nash)
var_order = 1:n_nash
transformed_rows = copy(payoffs);
row_coeffs = Array{Int64}(undef, (n_nash, n_nash)) .* 0;
for i in 1:n_nash
    row_coeffs[i, i] = 1
end
var_i = 1
add_row = rand(findall(transformed_rows[:, var_order[var_i]] .!= 0))
cf_new = sign(transformed_rows[add_row, var_order[var_i]])
old_row = transformed_rows[var_i, :]
new_row = cf_new .* transformed_rows[add_row, :]
old_coeff = row_coeffs[var_i, :]
new_coeff = [0 for i in 1:n_nash];
new_coeff[add_row] = cf_new;
transformed_rows[add_row, :] = old_row
transformed_rows[var_i, :] = new_row
row_coeffs[add_row, :] = old_coeff
row_coeffs[var_i, :] = new_coeff;
for i in (var_i + 1):n_nash
    cf0 = transformed_rows[i, var_order[var_i]]
    cf1 = transformed_rows[var_i, var_order[var_i]]
    gc = gcd(cf0, cf1)
    cf1 = div(cf1,gc)
    cf0 = div(cf0,gc)
    transformed_rows[i, :] = cf1 .* transformed_rows[i, :] .- cf0 .* transformed_rows[var_i, :]
    row_coeffs[i, :] = cf1 .* row_coeffs[i, :] .- cf0 .* row_coeffs[var_i, :]
end

In [236]:
for var_i in 2:n_nash
    println(var_i)
    cands = findall(transformed_rows[:, var_order[var_i]] .!= 0)
    cands = cands[cands .>= var_i]
    if length(cands) > 0
        avs = map(abs, transformed_rows[cands, var_order[var_i]])
        cands = cands[avs .== minimum(avs)]
        add_row = rand(cands)
        new_row = transformed_rows[add_row, :]
        new_coeff = row_coeffs[add_row, :]
        old_row = transformed_rows[var_i, :]
        old_coeff = row_coeffs[var_i, :]        
        cf_new = sign(new_row[var_order[var_i]])
        new_row = cf_new .* new_row
        new_coeff = cf_new .* new_coeff
        for i in 1:(var_i - 1)
            cf0 = new_row[var_order[i]]
            cf1 = transformed_rows[i, var_order[i]]
            # check what multiplier is needed
            gc = gcd(cf0, cf1)
            cf1 = div(cf1,gc)
            cf0 = div(cf0,gc)
            new_row = cf1 .* new_row .- cf0 .* transformed_rows[i, :]
            new_coeff = cf1 .* new_coeff
            new_coeff = new_coeff - cf0 .* row_coeffs[i, :]
        end
        transformed_rows[add_row, :] = old_row
        transformed_rows[var_i, :] = new_row
        row_coeffs[add_row, :] = old_coeff
        row_coeffs[var_i, :] = new_coeff
    end
    for i in (var_i + 1):n_nash
        cf0 = transformed_rows[i, var_order[var_i]]
        cf1 = transformed_rows[var_i, var_order[var_i]]
        gc = gcd(cf0, cf1)
        cf1 = div(cf1,gc)
        cf0 = div(cf0,gc)
        transformed_rows[i, :] = cf1 .* transformed_rows[i, :] .- cf0 .* transformed_rows[var_i, :]
        row_coeffs[i, :] = cf1 .* row_coeffs[i, :] .- cf0 .* row_coeffs[var_i, :]
    end
end

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [237]:
transformed_rows

40×40 Matrix{Int64}:
 1  1  -1  1   1  -1  -1   1  -1  -1  -1   0   1  -1  …                     1
 0  1   0  0   0   0   0   0   0   0   0  -1   0   0                        0
 0  0   1  0  -2   0   0   0   2   0   2  -1  -2   0                       -2
 0  0   0  1   1  -1   1   1  -1   1  -1   1   3  -1                        1
 0  0   0  0   1   0  -2   0  -2   0   0   1   0   0                        2
 0  0   0  0   0   1   0  -2   0   0   0  -1  -2   2  …                     0
 0  0   0  0   0   0   2  -2   2   0   0  -1   0   0                       -2
 0  0   0  0   0   0   0   2   0   2   0   0   0   0                        2
 0  0   0  0   0   0   0   0   1   0   0  -1  -2   0                       -2
 0  0   0  0   0   0   0   0   0   2   0   0   0   0                        2
 0  0   0  0   0   0   0   0   0   0   1   0   0   0  …                     0
 0  0   0  0   0   0   0   0   0   0   0   1  -2   0                        0
 0  0   0  0   0   0   0   0   0   0   0   

In [240]:
a = [0 for i in 1:n_nash]
for i in 1:n_nash
    a = a .+ payoffs[i, :] .* row_coeffs[19, i]
end
maximum(a .- transformed_rows[19, :])

0

In [242]:
minimum([transformed_rows[i,var_order[i]] for i in 1:n_nash])

1

In [243]:
# invertible because it includes dominated strats

## Using Power iterations

In [244]:
nash_env_df = DataFrame(CSV.File("spudsD_mxv10_nash.csv"))
nash_env = df_to_spuds(nash_env_df)
counts = [parse(Int, s.name[2:end]) for s in nash_env];
n_nash = length(nash_env)
payoffs = Array{Int64}(undef, (n_nash, n_nash))
for i in 1:n_nash
    for j in 1:n_nash
        payoffs[i, j] = eval_battle(nash_env[i], nash_env[j])
    end
end

In [245]:
payoffs2 = Array{Int64}(undef, (n_nash, n_nash))
for i in 1:n_nash
    for j in 1:n_nash
        payoffs2[i, j] = sum(payoffs[i, :] .* payoffs[:, j])
    end
end

In [308]:
v = [0 for i in 1:n_nash]
for i in 1:20
    v[rand(1:n_nash)] = 1
    v[rand(1:n_nash)] = -1
end
v = map(x -> div(x * 1000, maximum(map(abs, v))), v)
oldnorm = sum(v .* v)

39000000

In [309]:
for i in 1:20
    v2 = [sum(-payoffs2[i, :] .* v) for i in 1:n_nash]
    v2
    v = v2
    v[1:3]
    newnorm = sum(v .* v)
    println(newnorm/oldnorm)
    v = map(x -> div(x * 1000, maximum(map(abs, v))), v)
    oldnorm = sum(v .* v)
end

1.0247938051282052e7
5.527830366735122e8
7.644854497359102e8
9.113739109106829e8
9.800994722752857e8
1.0054357055565641e9
1.0139243173109868e9
1.0166675595219957e9
1.01754522158374e9
1.0178260663331027e9
1.0179153426548574e9
1.0179437452875867e9
1.0179526863417381e9
1.0179557716026e9
1.017956677986752e9
1.0179570276543334e9
1.0179571382431833e9
1.0179571528814958e9
1.0179571644794595e9
1.0179571655434585e9


In [310]:
v

407-element Vector{Int64}:
 -786
   58
  304
 -244
 -885
  591
  754
  346
  377
  341
  772
 -457
  299
    ⋮
 -388
 -943
 -941
 -673
 -680
  -67
 -981
 -858
 -883
  -81
 -371
 -999

In [311]:
# eigenvalue of payoff^2 is approx 31937

In [440]:
sqrt(31905)

178.61970775925036

In [454]:
maxv = 100000000
mult = 179
mult * mult

32041

In [464]:
v = [maxv for i in 1:n_nash];

In [465]:
#v = counts
#v = map(x -> div(x, maximum(v)), v .* maxv);

In [466]:
v2 = [sum(payoffs2[i, :] .* v) for i in 1:n_nash];
v = v .* mult .+ map(x -> div(x, mult), v2);
println(log(maximum(v))/log(2))
v = map(x -> div(x, maximum(v)), v .* maxv)

34.93550550394038


407-element Vector{Int64}:
 100000000
  92668536
  41892374
  89635297
  85184051
  14236164
  54504803
  60700501
  67457281
  50271189
  21934880
  22982232
  66352121
         ⋮
  22240924
  78750318
  78927144
  82766301
  82715293
  14817648
  49346255
  36778032
  85445889
  15120292
  21880472
  46248406

In [467]:
it_count = 0
while it_count < 10000
    it_count += 1
    if mod(it_count, 10000) == 0
        println(minimum(v))
    end
    v2 = [sum(payoffs2[i, :] .* v) for i in 1:n_nash];
    v = v .* mult .+ map(x -> div(x, mult), v2);
    #println(log(maximum(v))/log(2))
    v = map(x -> div(x, maximum(v)), v .* maxv);
    v[v .< 0] .= 0
end

0


In [475]:
# for k in v
#     println(k)
# end

In [469]:
w2 = v./sum(v)

407-element Vector{Float64}:
 0.09470113914778636
 0.047868883113619885
 0.01832183696701336
 0.0318155172337526
 0.011975595436892658
 0.01028974753904577
 0.010555194832077015
 0.00987954766080435
 0.0013511494498024334
 0.0012803234148451954
 0.014180004991450822
 0.0017879433019393344
 0.0
 ⋮
 0.0
 0.0
 0.003788314517146634
 0.0
 0.0
 0.0
 0.0
 0.0002365994320240465
 0.0
 0.0
 0.013164050223661978
 0.0005056141169669888

In [470]:
ebs = [sum(payoffs[i, :] .* w2) for i in 1:n_nash]

407-element Vector{Float64}:
  0.018213833158849474
  0.0035064290003821033
  0.01539604622005393
  0.003439526433619771
  0.008447950540307267
 -0.0014033591348260053
  4.0587014215974666e-5
 -0.0037855340917012795
 -0.003014969862683527
 -0.002853656889270557
 -0.0038871645132004954
  0.007580720123504415
  0.01748882964787575
  ⋮
 -0.021774317569457177
 -0.00428430699837619
 -0.013878975639407584
 -0.01234279815692682
  0.0015142001891187288
 -0.0034505827916152887
 -0.013847087871833787
 -0.023960963084334007
 -0.0012513249790267458
 -0.0034505827916152887
 -0.02010951605277449
 -0.0034646279175623122

In [471]:
maximum(ebs)

0.029649974920723486

In [472]:
# power iteration not working well due to inaccurate support

In [474]:
v[1:20]

20-element Vector{Int64}:
 100000000
  50547315
  19347008
  33595707
  12645672
  10865495
  11145795
  10432343
   1426751
   1351962
  14973426
   1887985
         0
   5783575
   7606842
    558242
  13954229
  20330936
   2282592
   5807038